In [1]:
import os
import string
import re

import gensim
from gensim.models import word2vec

import nltk
from nltk import corpus
from nltk.tokenize import TweetTokenizer, sent_tokenize

/usr/local/lib/python3.5/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [2]:
folder = 'questions'
files = [file for file in os.listdir(folder) if file.endswith('.txt')]

print(len(files))

127502


In [3]:
raw_questions = []

for file in files:
    with open(os.path.join(folder, file)) as f:
        raw_questions.append(f.read())

Let's save raw, unprocessed questions:

In [4]:
with open('raw_questions.txt', 'w') as f:
    f.write('\n'.join(raw_questions))

In [5]:
raw_questions[0]

'Можно ли есть такой творог?'

# Processing

## Уберём пунктуацию и стоп-слова

1\. Пунктуация

In [6]:
print(string.punctuation)
punctuation = '"#$%&\'()*+/:,-.?!;<=>@[\]^_`{|}~'

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [7]:
# questions = [''.join([ch for ch in q if ch not in punctuation]).lower()  for q in raw_questions]
# for q in questions[:5]: print(q)

2\. Уберём из предложений также стоп-слова из `nltk.corpus.stopwords.words('russian')`.

In [8]:
stopwords = list(set(corpus.stopwords.words('russian')))

print(len(stopwords))
print(str(stopwords[:15]))

151
['ничего', 'какая', 'эти', 'ему', 'не', 'только', 'раз', 'и', 'тебя', 'всех', 'почти', 'этой', 'разве', 'ну', 'куда']


In [9]:
def raw_question_to_clean(raw_question, stopwords = stopwords):
    """
    raw_question: str
    stopwords: list of str
    
    returns: str (question without stopwords and punctuation)
    e.g:
        raw_question = 'С чем будет связана моя будущая работа? кем лучше стать? 25.09.1999'
        returns 'связана будущая работа кем стать 25 09 1999'
    """
    
    letters_only = re.sub('[^a-zA-Zа-яА-я0-9]', ' ', raw_question) 
    
    words = letters_only.lower().split()                                                  
    meaningful_words = [w for w in words if not w in stopwords]   

    return(' '.join( meaningful_words ))   

In [10]:
questions = [raw_question_to_clean(q) for q in raw_questions]

Сравним почищенные (`questions`) и исходные, необработанные вопросы (`raw_questions`):

In [11]:
for q, rq in zip(questions[:5], raw_questions[:5]): print('before: {}\nafter:  {}\n'.format(rq, q))

before: Можно ли есть такой творог?
after:  творог

before: Возможная стоимость данных предметов
after:  возможная стоимость данных предметов

before: Можно ли есть такой творог? Кислый на вкус
after:  творог кислый вкус

before: Драка во сне
after:  драка сне

before: Что же это за военные в РФ, которых можно спокойно спеленать и вывести за пределы страны?
after:  это военные рф которых спокойно спеленать вывести пределы страны



Очевидно, что просто убирать пунктуацию не очень эффективно. Да и без стоп-слов вопросы с Ответов.Mailru кажутся полным бредом. 

In [12]:
# nltk.data.load('tokenizers/punkt/russian.pickle')

------

## Попробуем `TweetTokenizer()`

In [13]:
def tokenize_question(raw_question, tokenizer = TweetTokenizer(), stopwords = None):
    """
    raw_question: str
    tokenizer: tokenizer, default: TweetTokenizer()
    stopwords: list, default: None
    
    returns: list of str (tokenized question)
    e.g.:
        raw_question = Иисус это Вселенная?
        returns ['иисус', 'это', 'вселенная', '?']     
    """
    words = tokenizer.tokenize(raw_question.lower())
                     
    if stopwords:
        words = [w for w in words if not w in stopwords]   

    return words

In [14]:
tokens = [tokenize_question(q) for q in raw_questions]

In [15]:
print('sentenses: ', len(tokens))
print('words: ', len([token for sent in tokens for token in sent]))
print('unique words: ', len(set([token for sent in tokens for token in sent])))

sentenses:  127502
words:  1522827
unique words:  133062


In [17]:
for t in tokens[:5]: print(str(t))

['можно', 'ли', 'есть', 'такой', 'творог', '?']
['возможная', 'стоимость', 'данных', 'предметов']
['можно', 'ли', 'есть', 'такой', 'творог', '?', 'кислый', 'на', 'вкус']
['драка', 'во', 'сне']
['что', 'же', 'это', 'за', 'военные', 'в', 'рф', ',', 'которых', 'можно', 'спокойно', 'спеленать', 'и', 'вывести', 'за', 'пределы', 'страны', '?']


Окей, выглядит неплохо, но там наверняка много мусора.

* Добавим `reduce_len = True` (вместо 53893 уникальных слов мы получим 53881): 

In [18]:
tokens_reduce_len = [tokenize_question(q, TweetTokenizer(reduce_len = True)) 
                     for q in raw_questions]

In [19]:
print('sentenses: ', len(tokens_reduce_len))
print('words: ', len([token for sent in tokens_reduce_len for token in sent]))
print('unique words: ', len(set([token for sent in tokens_reduce_len for token in sent])))

sentenses:  127502
words:  1522820
unique words:  133027


In [20]:
for t in list(set([token for sent in tokens for token in sent])
              .difference(set([token for sent in tokens_reduce_len for token in sent])))[:10]: print(t)

0,0000
30000-45000
640000р
30000
10000
5256654
2024561111
0xc0000023
3200000
60000


* Добавим `strip_handles = True` к `reduce_len = True` (вместо 53881 уникальных слов мы получим 53878). 

(Потеряются слова `{'@zakladkis', '@kiska', '@instagram'}`. На мой взгляд, особой важности в них не было.)

In [21]:
tokens_reduce_strip = [tokenize_question(q, TweetTokenizer(reduce_len    = True, 
                                                           strip_handles = True)) 
                       for q in raw_questions]

In [22]:
print('sentenses: ', len(tokens_reduce_strip))
print('words: ', len([token for sent in tokens_reduce_strip for token in sent]))
print('unique words: ', len(set([token for sent in tokens_reduce_strip for token in sent])))

sentenses:  127502
words:  1522810
unique words:  133020


In [23]:
for t in (set([token for sent in tokens_reduce_len for token in sent])
          .difference(set([token for sent in tokens_reduce_strip for token in sent]))): print(t)

@yahoo
@yandex
@zakladkis
@instagram
@medi
@skypoint
@echo


------

In [24]:
def is_word(token):
    return not re.search(r'^[\w\d\-]+$', token) is None

In [25]:
def is_okay(token):
    token_is_word  = is_word(token)
    token_is_punct = token in [',', '-', ':']
    
    return token_is_word or token_is_punct

In [26]:
# token_list = tokens_reduce_strip[67]
# print(str(token_list))

# for token in token_list: print(token, '\t\t\t', re.search(r'^[\w\d\-]+$' ,token) is None)
# for token in token_list: print(token, '\t\t\t', is_word(token))
# for token in token_list: print(token, '\t\t', is_okay(token))

In [27]:
questions_tokenized = []

for token_list in tokens_reduce_strip:
    token_list_ok = [t for t in token_list if is_okay(t)]
    question = ' '.join(token_list_ok)
    questions_tokenized.append(question)

In [28]:
for q in questions_tokenized[:5]: print(str(q))

можно ли есть такой творог
возможная стоимость данных предметов
можно ли есть такой творог кислый на вкус
драка во сне
что же это за военные в рф , которых можно спокойно спеленать и вывести за пределы страны


------

А теперь попробуем повторить последовательность действий `tokenize (reduce, strip)`, `is_okay`, `join`, но не с цельными вопросами, как в прошлом случае, а с отдельными предложениями (т.е. если вопрос — «зачем? почему?», будем делать указанные выше действия для «зачем?» и для «почему?» по отдельности.

In [29]:
joined_questions = '\n'.join(raw_questions)

In [30]:
questions_sent_tokenized = sent_tokenize(joined_questions)

In [31]:
questions_sent_tokenized[:10]

['Можно ли есть такой творог?',
 'Возможная стоимость данных предметов\nМожно ли есть такой творог?',
 'Кислый на вкус\nДрака во сне\nЧто же это за военные в РФ, которых можно спокойно спеленать и вывести за пределы страны?',
 'кто знает сайты для девочек подростков где можно обсуждать все\nПатриарх Кирилл объявил Христа и апостолов «неудачниками»!',
 'если бы и правда было - нажал на кномпачку ...и секс получил.',
 'ты бы скока жал?)))',
 'В Киеве отметили 3-летие майдана факельными шествиями, зигами, нацилозунгами, драками поджогами зданий.',
 'ВОТ ГДЕ РОМАНТИКА!?',
 '㋡Чем торгуют на этом мультяшном рынке?',
 '㋡ςμ⇨⇨⇨\nкак не сойти с ума в росси и ?']

In [32]:
tokens_reduce_strip = [tokenize_question(q, TweetTokenizer(reduce_len    = True, 
                                                           strip_handles = True)) 
                       for q in questions_sent_tokenized]

------

Дополнительная обработка:

In [33]:
def remove_space_before(string):
    """
    string: str
    returns: str (without spaces before commas and colons)
    e.g.:
        string = 'дять , а 2 провода которые вышли от транса куда уйдет'
        returns 'дять, а 2 провода которые вышли от транса куда уйдет'
    """
    s = re.sub(' ,', ',', string)
    s = re.sub(' :', ':', s)
    return  s

------

Тут марковчейн для развлечения, пока вопросы парсятся.

In [75]:
import markovify
import pymarkovchain

# mv = markovify.Text('\n'.join(questions), state_size = 2)
# for _ in range(50): print(mv.make_sentence(), '\n')

In [76]:
mc = pymarkovchain.MarkovChain()
mc.generateDatabase('\n'.join(questions_tokenized))

In [91]:
for _ in range(50): print('- ' + (remove_space_before(mc.generateString())) + '?')

- как создать готический сайт?
- а она что на четвереньках, очень нужно?
- васкогда нить фоткали когда вы поняли, что нужно готовить сына путина к федеральному собранию по отношению ко мне мужчина, какие еще знаки о рождении детей вы хотели изменить в системе увеличить?
- срочно?
- как накрутить часы в skyrim?
- как выглядит грустный человек т е как его написать?
- въ способнъ прощать и давать еще один ребус от меня надо?
- почему так пишет?
- америкосы столько врут, что дяди нет?
- безбожное поколение потомков недобитых большевиков и коммунистов вновь готово громить православные храмы?
- за это отдать?
- что делать?
- научите?
- наш зато с колен без путина?
- а как ты подпитываешь себя в зеркале другое где правда подскажите?
- помогите пожалуйста я не из гдз?
- транзисторы разных частот?
- witcher enhanced edition ошибка при входе на карту придут ли они там размещаться?
- а кто тогда такие орбы-плазмоиды, если будет лобовое столкновение?
- поверхность земли 22 гр воздушный шар что вы 